# Calculate BIC curves

This notebook will fit 2 to 20 class GMM models to three UK-ESM historical ensemble members, caculate the BIC score for each, and save to model_19651995/BICs2-30.obj. This is required to reproduce Figure 2 from *Heuristic Methods for Determining the Number of Classes in Unsupervised Classification of Climate Models*, E. Boland et al. 2022 (doi to follow). This requires cluster_utils.py and input datafiles via the googleapi CMIP6 store (see cluster_utils.py for more info)

Please attribute any plots or code from this notebook using the DOI from Zenodo: to come

Updated Nov 2022
E Atkinson & E Boland [emmomp@bas.ac.uk](email:emmomp@bas.ac.uk)

In [70]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:39925")
client

<Client: 'tcp://127.0.0.1:39925' processes=1 threads=1, memory=8.00 GiB>

In [67]:
client.close()

2022-11-30 12:47:19,177 - distributed.client - ERROR - 
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/core.py", line 291, in connect
    comm = await asyncio.wait_for(
  File "/srv/conda/envs/notebook/lib/python3.9/asyncio/tasks.py", line 479, in wait_for
    return fut.result()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 461, in connect
    convert_stream_closed_error(self, e)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc.__class__.__name__}: {exc}") from exc
distributed.comm.core.CommClosedError: in <distributed.comm.tcp.TCPConnector object at 0x7f3b733c3640>: ConnectionRefusedError: [Errno 111] Connection refused

During ha

In [68]:
from dask_gateway import Gateway
gateway = Gateway()
from dask.distributed import Client

'''properly shutdown any previous clusters'''
clusters=gateway.list_clusters()
if clusters != []:
    print(f'found {len(clusters)} clusters')
    for cluster in clusters:
        cluster = gateway.connect(cluster.name)
        client=Client(cluster)
        client.close()
        cluster.shutdown()

In [71]:
import numpy as np
import os
import pickle
import cluster_utils as flt
from sklearn import metrics

### User options
Leave as is to recreate the paper

In [72]:
# Number of classes 
model_folder='model_20012017'
max_classes = 20 #max classes
#Time range
tslice=slice('2001-01', '2017-12') 
#Depth range
levSel=slice(5, 2000)
ids = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2']
ntrain=3000 #number of profiles per month to use in training dataset
npca=3

Uncomment the following three lines if you need to generate mask.npy:

In [73]:
#data = flt.retrieve_profiles(timeRange = slice('1995-01', '1995-02'),levSel=levSel)
#np.save('data/mask_3000', data['n'])
#mask=data['n']
mask = np.load('data/mask_3000.npy', allow_pickle=True)

### Fit 2-30 class models for each ensemble member
Saves each individual PCA model, GMM model and BIC/AIC score to \[model_folder\]

Saves all BICs/AICs to \[model_folder\]/\[BICs/AICs\]2-30.obj

In [77]:
BICs = {}
AICs = {}
SILs = {}
for m_id in ids:
    path_id = '{}/{}'.format(model_folder, m_id)
    if not os.path.isdir(path_id):
        os.makedirs(path_id)
    print('Starting {}'.format(m_id))
    options = {'memberId' : m_id}
    
    # Load training set
    [data,pca] = flt.generate_trainingset(timeRange = tslice, mask=mask, options=options,N=ntrain,n_components=npca,levSel=levSel)
    
    bic = np.zeros(max_classes-1)
    aic = bic.copy()
    sil = bic.copy()
    
    with open('{}/pca.obj'.format(path_id), 'wb') as file:
        pickle.dump(pca, file)
        
    print('Finished setup for {}'.format(m_id))
    
    for n_classes in range(2, max_classes+1):
        
        path_n = '{}/{}/{}'.format(model_folder, m_id, n_classes)
        
        if not os.path.isdir(path_n):
            os.makedirs(path_n)
            
        if os.path.exists('{}/gmm.obj'.format(path_n)):
            with open('{}/gmm.obj'.format(path_n), 'rb') as file:
                gmm=pickle.load(file)
        else:
            gmm = flt.train_gmm(data, n_classes)
            with open('{}/gmm.obj'.format(path_n), 'wb') as file:
                pickle.dump(gmm, file)
        
        if os.path.exists('{}/bic.obj'.format(path_n)):
            with open('{}/bic.obj'.format(path_n), 'rb') as file:
                bic[n_classes-2] = pickle.load(file)
        else:
            bic[n_classes-2] = gmm.bic(data)
            with open('{}/bic.obj'.format(path_n), 'wb') as file:
                pickle.dump(bic[n_classes-2],file)
        
        if os.path.exists('{}/aic.obj'.format(path_n)):
            with open('{}/aic.obj'.format(path_n), 'rb') as file:
                aic[n_classes-2] = pickle.load(file)
        else:        
            aic[n_classes-2] = gmm.aic(data)
            with open('{}/aic.obj'.format(path_n), 'wb') as file:
                pickle.dump(aic[n_classes-2],file)     

        if os.path.exists('{}/sil.obj'.format(path_n)):
            with open('{}/sil.obj'.format(path_n), 'rb') as file:
                sil[n_classes-2] = pickle.load(file)
        else:       # Calculate silhouette score for 10000 point sample        
            inds=np.random.randint(0,data.shape[0],10000)
            labels=flt.gmm_classify(data[inds,:],gmm)
            sil[n_classes-2]=metrics.silhouette_score(data[inds,:],labels,n_jobs=-1)
            sample_silhouette_values = metrics.silhouette_samples(data[inds,:],labels,n_jobs=-1)
            with open('{}/sil.obj'.format(path_n), 'wb') as file:
                pickle.dump(sil[n_classes-2],file)
            with open('{}/sil_vals.obj'.format(path_n), 'wb') as file:
                pickle.dump(sample_silhouette_values,file)
                
        
        print('Finished {} with {} classes'.format(m_id, n_classes))
        
    BICs[m_id] = bic
    AICs[m_id] = aic
    SILs[m_id] = sil
    
with open('{}/BICs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(BICs, file)
with open('{}/AICs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(AICs, file)
with open('{}/SILs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(SILs, file)

print('Done!')

Starting r1i1p1f2
Finished setup for r1i1p1f2
Finished r1i1p1f2 with 2 classes
Finished r1i1p1f2 with 3 classes
Finished r1i1p1f2 with 4 classes
Finished r1i1p1f2 with 5 classes
Finished r1i1p1f2 with 6 classes
Finished r1i1p1f2 with 7 classes
Finished r1i1p1f2 with 8 classes
Finished r1i1p1f2 with 9 classes
Finished r1i1p1f2 with 10 classes
Finished r1i1p1f2 with 11 classes
Finished r1i1p1f2 with 12 classes
Finished r1i1p1f2 with 13 classes
Finished r1i1p1f2 with 14 classes
Finished r1i1p1f2 with 15 classes
Finished r1i1p1f2 with 16 classes
Finished r1i1p1f2 with 17 classes
Finished r1i1p1f2 with 18 classes
Finished r1i1p1f2 with 20 classes
Starting r2i1p1f2
Finished setup for r2i1p1f2
Finished r2i1p1f2 with 2 classes
Finished r2i1p1f2 with 3 classes
Finished r2i1p1f2 with 4 classes
Finished r2i1p1f2 with 5 classes
Finished r2i1p1f2 with 6 classes
Finished r2i1p1f2 with 7 classes
Finished r2i1p1f2 with 8 classes
Finished r2i1p1f2 with 9 classes
Finished r2i1p1f2 with 10 classes
Finish